In [2]:
import ibmdata
import pandas as pd

In [67]:
DAYSBACK = 300
QUERY = f"""
WITH cte AS(
    SELECT 
        LEFT(lot_Id,5) AS lot_id_base, wafer_Id, 
        unitcell_X || '_' || unitcell_Y AS kerf_XY, family_Code,
        MAX(CASE 
            WHEN UCASE(tp.parm_Label) = 'CAPC_GF_ALL0304_CYLD_LT' THEN parmYield
            ELSE NULL
        END) AS CAPC_GF_ALL0304_cyld_LT,
        MAX(CASE 
            WHEN UCASE(tp.parm_Label) = 'SAIL2_H2_0P65TO1P05_PERFECTYIELD' THEN parmYield
            ELSE NULL
        END) AS SAIL2_H2_0P65TO1P05_PERFECTYIELD
    FROM 
        DMIW.ChipParmFactR cpfr
    INNER JOIN
        DMIW_SYSPRC.TestParm tp ON tp.testParmKey = cpfr.testParmKey
    INNER JOIN
        DMIW_SYSPRC.Geography g ON g.geographyKey = cpfr.geographyKey
    INNER JOIN
        DMIW_SYSPRC.TestedWafer tw ON tw.TestedWaferKey = cpfr.TestedWaferKey
    WHERE
        Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP'
        AND Level IN ('LT', 'H2')
        AND family_Code = 'Q6'
        AND cpfr.parmValue is not null and abs(parmValue) < 1e25
        AND (UCASE(tp.parm_Label) IN ('CAPC_GF_ALL0304_CYLD_LT', 'SAIL2_H2_0P65TO1P05_PERFECTYIELD'))
        AND unitcell_Y = 11
        AND unitcell_X IN (7, 8)
    GROUP BY
        LEFT(lot_Id,5), wafer_Id, unitcell_X || '_' || unitcell_Y, family_Code
)

SELECT 
    lot_id_base, kerf_xy, family_code, 
    AVG(capc_gf_all0304_cyld_lt) AS capc_gf_all0304_cyld_lt,
    AVG(sail2_h2_0p65to1p05_perfectyield) AS sail2_h2_0p65to1p05_perfectyield
FROM
    cte
GROUP BY
    lot_id_base, kerf_xy, family_code
"""

In [68]:
df = ibmdata.isdw.query(QUERY)

In [70]:
df

,lot_id_base,kerf_xy,family_code,capc_gf_all0304_cyld_lt,sail2_h2_0p65to1p05_perfectyield
0,A1AVP,7_11,Q6,100.0000000000000000000000000000,100.0000000000000000000000000000
1,A1AVP,8_11,Q6,0.0000000000000000000000000000,29.1666666666666666666666666666
2,A1AVR,7_11,Q6,100.0000000000000000000000000000,100.0000000000000000000000000000
3,A1AVR,8_11,Q6,0.0000000000000000000000000000,39.1304347826086956521739130434
4,A1AVT,7_11,Q6,95.6521739130434782608695652173,100.0000000000000000000000000000
...,...,...,...,...,...
89,AZAHX,8_11,Q6,None,45.4545454545454545454545454545
90,AZAKD,7_11,Q6,80.9523809523809523809523809523,88.0000000000000000000000000000
91,AZAKD,8_11,Q6,0.0000000000000000000000000000,32.0000000000000000000000000000
92,AZAKV,7_11,Q6,None,66.6666666666666666666666666666


In [66]:
df.to_csv("~/Downloads/CAPCSail.csv")